In [0]:
import tensorflow as tf
import numpy as np
import re

In [13]:
#reading data from the movie_conversations.txt file
movie_conversations = open('movie_conversations.txt','r')
conversations = []    #contains all the conversations as list elements ( list within a list )
for i in movie_conversations.readlines():
  a = i.split('+++$+++')
  b = a[4].split(', ') 
  conv= []
  for j in b:
    conv.append(re.sub(r'[^\w]', ' ',j).replace(" ",""))
  conversations.append(conv)
conversations[:5]

IndexError: ignored

In [0]:
#reading data from the movie_lines.txt
movie_lines = open('movie_lines.txt','rb')
code_vs_dialogue_list = dict()
for line in movie_lines:
  i = line.decode(encoding='utf-8',errors='ignore')
  a = i.split('+++$+++')
  conv_code = a[0].replace(" ","")
  dialogue = a[3].replace("\n","").strip()
  code_vs_dialogue_list.update({conv_code:dialogue})


In [0]:
def find_dialogue(x):
    return code_vs_dialogue_list[x]

In [0]:
questions = []
answers = []
for i in conversations:
  for k in range(len(i)-1):
    questions.append(find_dialogue(i[k]))
    answers.append(find_dialogue(i[k+1])) 

In [0]:
answers_with_tags = []
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( '<START> ' + answers[i] + ' <END>' )
    else:
        questions.pop( i )
        

In [0]:
#questions = questions[:1000]
#answers_with_tags = answers_with_tags[:1000]

In [0]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [0]:
tokenizer.fit_on_texts(questions + answers_with_tags)

In [0]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [0]:
upper_limit = 10
tokenized_questions = tokenizer.texts_to_sequences(questions)
tokenized_answers = tokenizer.texts_to_sequences( answers_with_tags )
final_tokens_questions = []
final_tokens_answers = []
for i in range(len(tokenized_questions)):
  if len(tokenized_questions[i])<=upper_limit:
    final_tokens_questions.append(tokenized_questions[i])
    final_tokens_answers.append(tokenized_answers[i])


AttributeError: ignored

In [13]:
#encoder_input_data_preprocessing
#maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
#padded_questions = tf.keras.preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
padded_questions = tf.keras.preprocessing.sequence.pad_sequences( final_tokens_questions , maxlen=upper_limit , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape )

(151057, 10)


In [14]:
# decoder_input_data
#tokenized_answers = tokenizer.texts_to_sequences( answers_with_tags )
#maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = tf.keras.preprocessing.sequence.pad_sequences( final_tokens_answers , maxlen=upper_limit , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape  )

(151057, 10)


In [0]:
for i in range(len(final_tokens_answers)) :
    final_tokens_answers[i] = final_tokens_answers[i][1:]

In [0]:
# decoder_output_data
#tokenized_answers = tokenizer.texts_to_sequences( answers_with_tags )

#maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = tf.keras.preprocessing.sequence.pad_sequences( final_tokens_answers , maxlen= upper_limit, padding='post' )
onehot_answers = tf.keras.utils.to_categorical( padded_answers , vocab_size )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape)

In [0]:
#Hyperparameters
##vocab_size = 1000
embedding_dim = 256
batch_size = 32
#input_length = 10
lstm_units = 1024
#encoder_input_training_data = tf.keras.layers.Input(shape=(None, ))
#decoder_input_training_data = tf.keras.layers.Input(shape=(None, ))
#decoder_output_training_data = tf.keras.layers.Input(shape=(None, ))

In [16]:
#training cell with training model created.
inference_encoder_inputs = tf.keras.layers.Input(shape=(None,)) #'''batch_size=batch_size''' )
embedded_encoder_inputs = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(inference_encoder_inputs)
encoder_output,state_h,state_c = tf.keras.layers.LSTM(lstm_units, return_state=True)(embedded_encoder_inputs) # '''input_shape=(batch_size,input_length,embedding_dim)'''
encoder_states = [state_h,state_c]

inference_decoder_inputs = tf.keras.layers.Input(shape=(None,)) #''',batch_size=batch_size''')
embedded_decoder_inputs = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True)(inference_decoder_inputs)
decoder_lstm_layer = tf.keras.layers.LSTM(lstm_units, return_state = True, return_sequences=True) #''',input_shape=(batch_size,input_length,embedding_dim)'''
decoder_output,_,_ = decoder_lstm_layer(embedded_decoder_inputs, initial_state = encoder_states)
dense_layer = tf.keras.layers.Dense(vocab_size, activation=tf.keras.activations.softmax)
decoder_output = dense_layer(decoder_output)

training_model = tf.keras.models.Model([inference_encoder_inputs,inference_decoder_inputs],decoder_output)
training_model.compile(optimizer = tf.keras.optimizers.RMSprop(), loss = 'categorical_crossentropy')

training_model.summary()

# Some important points - 
# raw inputs and outputs are used to define the model ( without embedding ). 
# NEVER overwrite tensors, because the LSTM layer requires brand new embedded layer


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    593152      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    593152      input_2[0][0]                    
______________________________________________________________________________________________

In [17]:
training_model.fit([encoder_input_data,decoder_input_data],decoder_output_data,batch_size=32, epochs = 1)
training_model.save('trainingmodel.h5')

32/32 [==============================] - 224s 7s/step - loss: 0.5335


In [0]:
def make_inference_models():
  inference_encoder_model = tf.keras.models.Model(inference_encoder_inputs,encoder_states)

  decoder_state_input_h = tf.keras.layers.Input(shape=(lstm_units, ))
  decoder_state_input_c = tf.keras.layers.Input(shape=(lstm_units, ))
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]

  decoder_output,state_h,state_c = decoder_lstm_layer(embedded_decoder_inputs, initial_state = decoder_states_inputs)
  decoder_states = [state_h,state_c]
  decoder_output = dense_layer(decoder_output)

  inference_decoder_model = tf.keras.models.Model([inference_decoder_inputs]+decoder_states_inputs, [decoder_output]+decoder_states)

  return inference_encoder_model,inference_decoder_model


In [0]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return tf.keras.preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [21]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : How are you 
 you end
Enter question : hi
 you end
Enter question : hello


KeyError: ignored